<a href="https://colab.research.google.com/github/staaason/imbd-spark-project/blob/develop/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 62.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=338a5ad0c1a979827043d6f37465e91b05f9e9f0bde76d7bfa98d75ff77717aa
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark import SparkConf
from pyspark.sql import  SparkSession, Window
import pyspark.sql.types as t
import pyspark.sql.functions as f
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/title.akas.tsv.gz", sep=r'\t', header=True, nullValue='null')

In [ ]:
df.show()

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       1|          Карменсіта|    UA|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       2|          Carmencita|    DE|      \N|         \N|       literal title|              0|
|tt0000001|       3|Carmencita - span...|    HU|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       4|          Καρμενσίτα|    GR|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       5|          Карменсита|    RU|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       6|          Carmencita|    US|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       7|          Carmenc

In [ ]:
df.printSchema()


root
 |-- titleId: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- types: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- isOriginalTitle: string (nullable = true)



In [6]:
title_akas_schema = t.StructType([t.StructField('titleId', t.StringType(), True),
                                  t.StructField('ordering', t.IntegerType(), True),
                                  t.StructField('title', t.StringType(), True),
                                  t.StructField('region', t.StringType(), True),
                                  t.StructField('language', t.StringType(), True),
                                  t.StructField('types', t.StringType(), True),
                                  t.StructField('attributes',  t.StringType(), True),
                                  t.StructField('isOriginalTitle', t.StringType(), True)])

title_akas_df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/title.akas.tsv.gz", sep=r'\t', header=True, nullValue='\\N', schema=title_akas_schema)
title_akas_df.show(30, truncate=False)

+---------+--------+-------------------------+------+--------+-----------+---------------------+---------------+
|titleId  |ordering|title                    |region|language|types      |attributes           |isOriginalTitle|
+---------+--------+-------------------------+------+--------+-----------+---------------------+---------------+
|tt0000001|1       |Карменсіта               |UA    |null    |imdbDisplay|null                 |0              |
|tt0000001|2       |Carmencita               |DE    |null    |null       |literal title        |0              |
|tt0000001|3       |Carmencita - spanyol tánc|HU    |null    |imdbDisplay|null                 |0              |
|tt0000001|4       |Καρμενσίτα               |GR    |null    |imdbDisplay|null                 |0              |
|tt0000001|5       |Карменсита               |RU    |null    |imdbDisplay|null                 |0              |
|tt0000001|6       |Carmencita               |US    |null    |imdbDisplay|null                 |

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/title.basics.tsv.gz", sep=r'\t', header=True, nullValue='\\N')
df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [30]:
title_basics_schema =  t.StructType([t.StructField('tconst', t.StringType(), True),
                                  t.StructField('titleType', t.StringType(), True),
                                  t.StructField('primaryTitle', t.StringType(), True),
                                  t.StructField('originalTitle', t.StringType(), True),
                                  t.StructField('isAdult', t.IntegerType(), True),
                                  t.StructField('startYear', t.DateType(), True),
                                  t.StructField('endYear',  t.DateType(), True),
                                  t.StructField('runtimeMinutes', t.IntegerType(), True),
                                  t.StructField('genres', t.StringType(), True)])

title_basics_df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/title.basics.tsv.gz", sep=r'\t', header=True, nullValue='\\N', dateFormat="MM/dd/yyyy", schema=title_basics_schema)
title_basics_df.show()

+---------+---------+--------------------+--------------------+-------+----------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult| startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+----------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|1894-01-01|   null|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|1892-01-01|   null|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|1892-01-01|   null|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|1892-01-01|   null|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|1893-01-01|   null|             1|      

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/name.basics.tsv.gz", sep=r'\t', header=True, nullValue='\\N')
df.printSchema()

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- deathYear: string (nullable = true)
 |-- primaryProfession: string (nullable = true)
 |-- knownForTitles: string (nullable = true)



In [26]:
name_basics_schema =  t.StructType([t.StructField('nconst', t.StringType(), True),
                                  t.StructField('primaryName', t.StringType(), True),
                                  t.StructField('birthYear', t.DateType(), True),
                                  t.StructField('deathYear', t.DateType(), True),
                                  t.StructField('primaryProfession', t.StringType(), True),
                                  t.StructField('knownForTitles',  t.StringType(), True)])


name_basics_df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/name.basics.tsv.gz", sep=r'\t', header=True, nullValue='\\N', dateFormat="MM/dd/yyyy", schema=name_basics_schema)
name_basics_df.show()

+---------+-------------------+----------+----------+--------------------+--------------------+
|   nconst|        primaryName| birthYear| deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+----------+----------+--------------------+--------------------+
|nm0000001|       Fred Astaire|1899-01-01|1987-01-01|soundtrack,actor,...|tt0072308,tt00504...|
|nm0000002|      Lauren Bacall|1924-01-01|2014-01-01|  actress,soundtrack|tt0037382,tt00383...|
|nm0000003|    Brigitte Bardot|1934-01-01|      null|actress,soundtrac...|tt0054452,tt00491...|
|nm0000004|       John Belushi|1949-01-01|1982-01-01|actor,soundtrack,...|tt0072562,tt00787...|
|nm0000005|     Ingmar Bergman|1918-01-01|2007-01-01|writer,director,a...|tt0050986,tt00509...|
|nm0000006|     Ingrid Bergman|1915-01-01|1982-01-01|actress,soundtrac...|tt0038109,tt00387...|
|nm0000007|    Humphrey Bogart|1899-01-01|1957-01-01|actor,soundtrack,...|tt0034583,tt00432...|
|nm0000008|      Marlon Brando|1924-01-0

In [ ]:
title_crew_schema =  t.StructType([t.StructField('tconst', t.StringType(), True),
                                  t.StructField('directors', t.StringType(), True),
                                  t.StructField('writers ', t.StringType(), True)])

title_crew_df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/title.crew.tsv.gz", sep=r'\t', header=True, nullValue='\\N', schema=title_crew_schema)
title_crew_df.show()

+---------+-------------------+---------+
|   tconst|          directors| writers |
+---------+-------------------+---------+
|tt0000001|          nm0005690|     null|
|tt0000002|          nm0721526|     null|
|tt0000003|          nm0721526|     null|
|tt0000004|          nm0721526|     null|
|tt0000005|          nm0005690|     null|
|tt0000006|          nm0005690|     null|
|tt0000007|nm0005690,nm0374658|     null|
|tt0000008|          nm0005690|     null|
|tt0000009|          nm0085156|nm0085156|
|tt0000010|          nm0525910|     null|
|tt0000011|          nm0804434|     null|
|tt0000012|nm0525908,nm0525910|     null|
|tt0000013|          nm0525910|     null|
|tt0000014|          nm0525910|     null|
|tt0000015|          nm0721526|     null|
|tt0000016|          nm0525910|     null|
|tt0000017|nm1587194,nm0804434|     null|
|tt0000018|          nm0804434|     null|
|tt0000019|          nm0932055|     null|
|tt0000020|          nm0010291|     null|
+---------+-------------------+---

In [ ]:
title_episode_schema =  t.StructType([t.StructField('tconst', t.StringType(), True),
                                  t.StructField('parentTconst', t.StringType(), True),
                                  t.StructField('seasonNumber', t.IntegerType(), True),
                                  t.StructField('episodeNumber', t.IntegerType(), True)])

title_episode_df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/title.episode.tsv.gz", sep=r'\t', header=True, nullValue='\\N', schema=title_episode_schema)
title_episode_df.show()

+---------+------------+------------+-------------+
|   tconst|parentTconst|seasonNumber|episodeNumber|
+---------+------------+------------+-------------+
|tt0041951|   tt0041038|           1|            9|
|tt0042816|   tt0989125|           1|           17|
|tt0042889|   tt0989125|        null|         null|
|tt0043426|   tt0040051|           3|           42|
|tt0043631|   tt0989125|           2|           16|
|tt0043693|   tt0989125|           2|            8|
|tt0043710|   tt0989125|           3|            3|
|tt0044093|   tt0959862|           1|            6|
|tt0044668|   tt0044243|           2|           16|
|tt0044901|   tt0989125|           3|           46|
|tt0045519|   tt0989125|           4|           11|
|tt0045960|   tt0044284|           2|            3|
|tt0046135|   tt0989125|           4|            5|
|tt0046150|   tt0341798|        null|         null|
|tt0046855|   tt0046643|           1|            4|
|tt0046864|   tt0989125|           5|           20|
|tt0047810| 

In [ ]:
title_principals_schema =  t.StructType([t.StructField('tconst', t.StringType(), True),
                                  t.StructField('ordering', t.IntegerType(), True),
                                  t.StructField('nconst', t.StringType(), True),
                                  t.StructField('category', t.StringType(), True),
                                  t.StructField('job', t.StringType(), True),
                                  t.StructField('characters', t.StringType(), True)])

title_principals_df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/title.principals.tsv.gz", sep=r'\t', header=True, nullValue='\\N', schema=title_principals_schema)
title_principals_df.show()

+---------+--------+---------+---------------+--------------------+--------------+
|   tconst|ordering|   nconst|       category|                 job|    characters|
+---------+--------+---------+---------------+--------------------+--------------+
|tt0000001|       1|nm1588970|           self|                null|      ["Self"]|
|tt0000001|       2|nm0005690|       director|                null|          null|
|tt0000001|       3|nm0374658|cinematographer|director of photo...|          null|
|tt0000002|       1|nm0721526|       director|                null|          null|
|tt0000002|       2|nm1335271|       composer|                null|          null|
|tt0000003|       1|nm0721526|       director|                null|          null|
|tt0000003|       2|nm1770680|       producer|            producer|          null|
|tt0000003|       3|nm1335271|       composer|                null|          null|
|tt0000003|       4|nm5442200|         editor|                null|          null|
|tt0

In [ ]:
title_ratings_schema =  t.StructType([t.StructField('tconst', t.StringType(), True),
                                  t.StructField('averageRating ', t.FloatType(), True),
                                  t.StructField('numVotes ', t.IntegerType(), True)])

title_ratings_df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/title.ratings.tsv.gz", sep=r'\t', header=True, nullValue='\\N', schema=title_ratings_schema)
title_ratings_df.show()

+---------+--------------+---------+
|   tconst|averageRating |numVotes |
+---------+--------------+---------+
|tt0000001|           5.7|     1925|
|tt0000002|           5.8|      261|
|tt0000003|           6.5|     1741|
|tt0000004|           5.6|      176|
|tt0000005|           6.2|     2554|
|tt0000006|           5.1|      175|
|tt0000007|           5.4|      797|
|tt0000008|           5.4|     2069|
|tt0000009|           5.3|      200|
|tt0000010|           6.9|     6992|
|tt0000011|           5.3|      357|
|tt0000012|           7.4|    12000|
|tt0000013|           5.7|     1850|
|tt0000014|           7.1|     5381|
|tt0000015|           6.2|     1035|
|tt0000016|           5.9|     1460|
|tt0000017|           4.6|      318|
|tt0000018|           5.3|      583|
|tt0000019|           5.1|       31|
|tt0000020|           4.8|      351|
+---------+--------------+---------+
only showing top 20 rows



In [35]:
def showAvailableInUA():
  titles_in_ua_df = title_akas_df.select(f.column('title')).where(f.col('region') == 'UA')
  titles_in_ua_df.show()
  path_to_save = '/content/drive/MyDrive/Colab Notebooks/output/titles_available_in_ukraine'
  titles_in_ua_df.write.options(header='True', delimiter=',').mode("overwrite").csv(path_to_save)
showAvailableInUA()
  

+--------------------+
|               title|
+--------------------+
|          Карменсіта|
|        Бідний П'єро|
|    Ковальська сцена|
|   Чхання Фреда Отта|
|Вихід робітників ...|
|Прибуття потяга н...|
|Прибуття делегаті...|
| Политий поливальник|
|     Навколо кабінки|
|              Ковалі|
|     Морське купання|
|      Партія в карти|
|Площа Кордельє в ...|
|Виловлювання черв...|
|  Сніданок немовляти|
|Стрибок через бре...|
|       Вольтижування|
|        Гра в сніжки|
|    Зруйнування муру|
|       Замок диявола|
+--------------------+
only showing top 20 rows



In [29]:
def getPeopleBornIn19Century():
  people_born_df = name_basics_df.select(f.column('primaryName').alias('name_surname')).where(f.year(name_basics_df.birthYear) >= 1800).where(f.year(name_basics_df.birthYear) < 1900)
  path_to_save = '/content/drive/MyDrive/Colab Notebooks/output/people_born_in_19_century'
  people_born_df.write.options(header='True', delimiter=',').mode("overwrite").csv(path_to_save)

getPeopleBornIn19Century()

In [34]:
def getMoviesLastTwoHours():
  movies_df = title_basics_df.select(f.column('originalTitle').alias('title')).where(f.col('runtimeMinutes') > 120).where(f.col('titleType') == 'movie')
  movies_df.show()
  path_to_save = '/content/drive/MyDrive/Colab Notebooks/output/get_movies_last_two_hours'
  movies_df.write.options(header='True', delimiter=',').mode("overwrite").csv(path_to_save)

getMoviesLastTwoHours()

+--------------------+
|               title|
+--------------------+
|What Happened to ...|
|The Adventures of...|
|            Atlantis|
|            Germinal|
|Les misérables - ...|
|The Active Life o...|
|The Beloved Adven...|
|             Cabiria|
|   L'enfant de Paris|
|The Exploits of E...|
|The Hazards of Helen|
|Lucille Love: The...|
|      The Master Key|
|The Perils of Pau...|
|The Port of Missi...|
|El signo de la tribu|
|  The Trey o' Hearts|
|The Birth of a Na...|
|       The Black Box|
|     The Broken Coin|
+--------------------+
only showing top 20 rows



In [36]:
def getNamesFilmsCharacter():
  names_df = name_basics_df.select("primaryName").where(name_basics_df.primaryProfession.contains('actor') | name_basics_df.primaryProfession.contains('actress'))
  names_df.show()

getNamesFilmsCharacter()

+-------------------+
|        primaryName|
+-------------------+
|       Fred Astaire|
|      Lauren Bacall|
|    Brigitte Bardot|
|       John Belushi|
|     Ingmar Bergman|
|     Ingrid Bergman|
|    Humphrey Bogart|
|      Marlon Brando|
|     Richard Burton|
|       James Cagney|
|        Gary Cooper|
|        Bette Davis|
|          Doris Day|
|Olivia de Havilland|
|         James Dean|
|   Marlene Dietrich|
|       Kirk Douglas|
|   Federico Fellini|
|        Henry Fonda|
|      Joan Fontaine|
+-------------------+
only showing top 20 rows

